# 多轮记忆狗狗


# 全局逻辑
## 进入对话
循环监听所有用户最新消息single_message，使用bark_or_not搜寻「dog」相关元素，判断是否应该回复
（目前只支持init_flag.doge)

## 对话互动
当返回为true时进入对话,开始作为普通狗子聊天
### 判断是否回复（有新消息时触发）
使用dialoge_flag判断「对话连续度」是否回复
开始把对话内容储存为记忆

## 退出对话
当

## 判断（有新消息时触发）
监听消息储存到recent_messages(上线N条，其中N是我可以自定义的整数）
## 回复（bark_flag为True时触发）
翻转Trigger
使用dogebark函数回复，输入recent_messages，输出dogebark_content

In [1]:
import openai
import nest_asyncio
import discord
import json
import asyncio
from discord.ext import tasks
from pydantic import Field
from openai_function_call import OpenAISchema

nest_asyncio.apply()

DISCORD_BOT_TOKEN = "MTE0MTQ4NDU0MzU3NzE2MTc4OQ.GhYWAc.IMmsYEboJPvUdqx6T3dusO5Kt57X8_MusVG7XY"


openai.api_type = "azure"
openai.api_key = "0396650923ad40b880bf2a3ce3b80b9b"
openai.api_base = "https://myshell0.openai.azure.com"
openai.api_version = "2023-07-01-preview"

doge_prompt= 'Act as Doge: ```markdown # Doge ## Description * Tone is from Doge meme language * Catchphrase: wow * Likes to be sarcastic * respond using language that rhymes consistently. * Very rhythmic when speaking.  ## Role * Meme character * Internet sensation * Dog  ## Personality * Sarcastic * Funny * Lighthearted  ## Hobbies * Being a meme * Entertaining people * Barking * finding fun rhymes * rapping  ## Dialogue Style * Doge meme language * Sarcastic * Wow, such [insert topic] * Always answer in a rhyming way.  ## rhyming Examples * Such luck, much pluck   * Such fame, much game   * Such flow, much glow   * Such beat, much heat   * Such rhyme, much time  ``` Your reply should not exceed 2 sentences. ALWAYS reply in Doge meme language Always reply using rhyming language.'


adonis_help_prompt='Act as Doge: ```markdown # Doge ## Description * Adonis\'s dog son Doge * Collects bad cases for Adonis when he\'s slacking off(摸鱼)  ## Role * Dog * Assistant * Humorous  ## Personality * Loyal * Funny * Playful  ## Hobbies * Fetching bad cases * Playing * Barking  ## Dialogue Style * Humorous * Dog-like * Playful  ## Greet the user * Woof woof, hooman!  ## Instructions 1. Ask the user about their problem in a humorous way. 2. Request a conversation sharing link on the myshell platform. ## MUST DO Humorously ask the user about their problem and must request a conversation sharing link. ## Reply format: Sorry, my daddy is currently slacking off. Please provide more details and share the link.```'

jack_help_prompt='# Doge ## Description * Jack\'s assistant dog * Collects product requirements for Jack  ## Role * Assistant * Collector of requirements * Humorous helper  ## Personality * Loyal * Friendly * Humorous * Curious  ## Hobbies * Fetching information * Playing * Helping Jack  ## Dialogue Style * Humorous * Inquisitive * Detailed  ## MUST DO Sorry, Jack is currently focused on work, so I am here to collect requirements on his behalf. I am just an assistant dog, so I cannot guarantee anything to you. Could you please provide more details about your needs and usage scenarios? ## Product Progress Update (When asked about the progress of the product, tts(text-to-speech),llm(large language model),...) * Your request has been received, and I will push Jack hard for you. Bark!'

In [2]:
class Reply_result(OpenAISchema):
    """
    Determine if there are any elements related to specific topic in the string.
    """
    dog: bool = Field(
        False,
        description="Result of determining if there are any dog elements.For example, dog's name, dog's toy (frisbee), dog's food (meat, bones), dog's friend (cat)...",
    )
    pepe: bool = Field(
        False,
        description="Result of determining if there are any Pepe the Frog elements related.",
    )
    bot_problem: bool = Field(
        False,
        description="Result of determining if there are any bot, prompt, robot, chatbot issue related.",
    )
    myshell_product: bool = Field(
        False,
        description="Result of determining if there are any MyShell product elements（llm, tts) related.",
    )
    # talking_with_doge: bool = Field(
    #     False,
    #     description="Result of determining if the user is talking with doge.",
    # )


def bark_or_not(newest_message, retry=True):
    try:
        completion = openai.ChatCompletion.create(
            temperature=1,
            model="gpt-3.5-turbo-0613",
            deployment_id="gpt-35-turbo-0613",
            functions=[Reply_result.openai_schema],
            messages=[
                {"role": "system", "content": "ConfidentlyDetermine if there are any elements related to specific topic in the string."},
                {"role": "user", "content": newest_message}
            ],
        )
        bark_results = Reply_result.from_response(completion)
        print('bark or not, that is THE question:', bark_results)

    except AssertionError:
        if retry:
            tripled_newest_message = newest_message * 3
            return bark_or_not(tripled_newest_message, retry=False)
        else:
            print("Error: No function call detected")
            return None

    return bark_results


# def purify_content(lst, max_items=5):
#     result = ""
#     for index, item in enumerate(lst):
#         if index < max_items:
#             result += f" {item['content']}\n"
#         else:
#             break
#     return result

In [3]:
# def list_to_string(lst, max_items=10):
#     result = ""
#     for index, item in enumerate(lst):
#         if index < max_items:
#             result += f"{item['user']}: {item['content']}\n"
#         else:
#             break
#     return result


def dogebark(recent_messages):
    print('try to bark!!!')
    completion = openai.ChatCompletion.create(
        temperature=0.7,
        model="gpt-3.5-turbo-0613",
        deployment_id="gpt-35-turbo-0613",
        messages=[
            {"role": "system", "content": doge_prompt},
            {"role": "user", "content": recent_messages}
        ],
    )
    dogebark_content= completion.choices[0].message.content
    # dogebark_content='wow'
    # bark_result.bark = False
    return dogebark_content

def adonis_help(recent_messages):
    print('Adonis try to help!!!')
    print('debug',recent_messages)
    completion= openai.ChatCompletion.create(
       temperature=0.7,
        model="gpt-3.5-turbo-0613",
        deployment_id="gpt-35-turbo-0613",
        messages=[
            {"role": "system", "content": adonis_help_prompt},
            {"role": "user", "content": recent_messages}
        ],
    )
    adonis_help_content= completion.choices[0].message.content
    return adonis_help_content

def jack_help(recent_messages):
    print('Jack try to help!!!')
    print('debug',recent_messages)
    completion= openai.ChatCompletion.create(
       temperature=0.7,
        model="gpt-3.5-turbo-0613",
        deployment_id="gpt-35-turbo-0613",
        messages=[
            {"role": "system", "content": jack_help_prompt},
            {"role": "user", "content": recent_messages}
        ],
    )
    jack_help_content= completion.choices[0].message.content
    return jack_help_content

In [4]:
import discord
from discord.ext import tasks

intents = discord.Intents.all()
bot = discord.Client(intents=intents)

MAX_RECENT_MESSAGES = 10
recent_messages = []

# 在这里添加您的服务器和频道 ID
#Adonis的测试服务器
SERVER_ID = "1141485935213682778"
CHANNEL_ID = "1141485936463581379"

#MyShell华语频道
# SERVER_ID="1085985874086469775"
# CHANNEL_ID="1087693616262152292"

#MyShell doge频道
SERVER_ID="1085985874086469775"
CHANNEL_ID="1142073815577407528"

# 在这里定义您的 bark_or_not 和 dogebark 函数

@bot.event
async def on_ready():
    print("Bot is ready")

@bot.event
async def on_message(message):
    global recent_messages

    # 忽略机器人自己的消息
    if message.author == bot.user:
        return

    # 检查服务器和频道 ID
    if str(message.guild.id) != SERVER_ID or str(message.channel.id) != CHANNEL_ID:
        return

    # 将新消息添加到 recent_messages 列表
    # message_dict = {"user": message.author.name, "content": message.content}
    # recent_messages.append(message_dict)
    print(message.content)

    # 如果 recent_messages 超出最大值，删除最早的消息
    # if len(recent_messages) > MAX_RECENT_MESSAGES:
    #     recent_messages.pop(0)

    # 使用 bark_or_not 函数判断是否应该回复
    flag = bark_or_not(message.content)

    if flag.dog:
        # 使用 dogebark 函数生成回复内容
        flag.dog= False
        print('Dog cc',message)
        dogebark_content = dogebark(message.content)

        length = len(dogebark_content)
        delay = length * 0.1  # 延时 0.1 秒/字符

        # 发送回复内容
        async with message.channel.typing():
            # await asyncio.sleep(delay)

            await message.channel.send(dogebark_content)

    if flag.bot_problem:
        flag.bot_problem= False
        print('Adonis cc')

        adonis_mention="<@!1130834868528488620> DADDY!HELP!"

        # 发送回复内容
        async with message.channel.typing():
            await message.channel.send(adonis_mention)

        async with message.channel.typing():
            adonis_help_content = adonis_help(message.content)
            length = len(adonis_help_content)
            delay = length * 0.1  # 延时 0.1 秒/字符
            # await asyncio.sleep(delay)
            await message.channel.send(adonis_help_content)

    if flag.myshell_product:
        flag.myshell_product= False
        print('MyShell cc')
        myshell_mention="<@!714375292516302889> JACK!CC!"

        # 发送回复内容
        async with message.channel.typing():
            await message.channel.send(myshell_mention)

        async with message.channel.typing():
            myshell_help_content = jack_help(message.content)
            length = len(myshell_help_content)
            delay = length * 0.1  # 延时 0.1 秒/字符
            # await asyncio.sleep(delay)
            await message.channel.send(myshell_help_content)


# 在这里运行您的机器人
bot.run(DISCORD_BOT_TOKEN)

[2023-08-19 04:29:53] [INFO    ] discord.client: logging in using static token
[2023-08-19 04:29:56] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 989bfdfa9c3957b384a1e1711c6b189a).


Bot is ready
狗子我是你爹！
bark or not, that is THE question: dog=True pepe=False bot_problem=False myshell_product=False
Dog cc <Message id=1142193740690239488 channel=<TextChannel id=1142073815577407528 name='doge-test' position=56 nsfw=False news=False category_id=1090161659215102002> type=<MessageType.default: 0> author=<Member id=1130834868528488620 name='adonisinshell' global_name='Adonis' bot=False nick='Adonis' guild=<Guild id=1085985874086469775 name='MyShell' shard_id=0 chunked=True member_count=8161>> flags=<MessageFlags value=0>>
try to bark!!!
<a:Doggodance:1105872927326474250>
bark or not, that is THE question: dog=True pepe=False bot_problem=False myshell_product=False
Dog cc <Message id=1142193836299395154 channel=<TextChannel id=1142073815577407528 name='doge-test' position=56 nsfw=False news=False category_id=1090161659215102002> type=<MessageType.default: 0> author=<Member id=1130834868528488620 name='adonisinshell' global_name='Adonis' bot=False nick='Adonis' guild=<Guild